IMPORTANT NOTE
- if mo ask si maam why this is similar to mine and mag suspect siya na i made it, kay just tell her na youre using mediapipe instead of tensorflow
- 

flaw/weaknesses observed:

evident luminance affecting performance, resulting to erratic tracking which causing it to abruptly stop collecting and considering it as one execution
- should always perform/collect data at a well lit environment
- should apply frame patience mechanism wherein it ignores certain amount of frames before it consider as an execution [APPLIED!]

overload of data
- apply frame buffering(less frame already is sufficient enough to resemble the movement)
- eliminate z coordinate(pose estimation is positional dependant)
- experiment on lessening decimal places (this would sacrifice a certain amount of accuracy, should heavily consider how much of it)


future planned features
 - real-time frame by frame visualization every after execution to check
 - easy-delete button wherein it deletes the current execution collected in case its not good or intended 


How data collection works?
1) show your hands on the camera----------------> data collection starts
2) perform the action you want -----------------> coordinates are collected(coordinates_frame are generated and are compiled in coordinates_movemenet )
3) remove your hand from the camera ------------> data collection stops(coordinates_movemenet is compiled in execution)
4) repeat...

coordinates
- individual X and Y coordinates tracked and collected 
- how it looks like: 0.435. 0.454....etc


coordinates_frame
- compilation of coordinates
- how it looks like: [coordinates, coordinates, coordinates, coordinates..., coordinates,]

execution
- compilation of coordinates_frame, which resembles a movement 
- how it looks like:
[
[coordinates, coordinates, coordinates, coordinates..., coordinates,],
[coordinates, coordinates, coordinates, coordinates..., coordinates,],
...
[coordinates, coordinates, coordinates, coordinates..., coordinates,],
]

datasets
- compilation of executions
- how it looks like:
[
execution,
execution,
execution,
...
execution,
]


In [1]:
import cv2
import os
import numpy as np
import tkinter as tk
from tkinter import simpledialog
from threading import Thread
import mediapipe as mp
import os

import random




In [2]:

# Globals
detection_mode = "Multiple"
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
label_map = {}
trained = False

# MediaPipe for hand tracking
mp_hands = mp.solutions.hands
hands_detector = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

known_folder = os.path.join(os.getcwd(), "datasets/")
parent_folder = os.path.dirname(known_folder)
file_path = os.path.join(parent_folder, "output.txt")

cap = cv2.VideoCapture(1)



In [3]:

def pose_estimation():
    frame_buffer= 8
    frame_buffer_ctr = 0

    frame_patience = 8
    frame_patience_ctr = 0
    
    coordinates_frame = []
    coordinates_movemenet = []
    execution = []

    while True:        
        frame_buffer_ctr+=1
        if frame_buffer_ctr>=frame_buffer:   
            frame_buffer_ctr = 0         
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands_detector.process(rgb)

            if results.multi_hand_landmarks:    
                frame_patience_ctr=0

                for hand_landmarks in results.multi_hand_landmarks:            
                    mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)             
                    for idx, landmark in enumerate(hand_landmarks.landmark):
                        coordinates_frame.append(f"{landmark.x:.3f}")
                        coordinates_frame.append(f"{landmark.y:.3f}")     
                coordinates_movemenet.append(coordinates_frame)
                coordinates_frame=[]
            else:
                if frame_patience_ctr<=frame_patience:
                    frame_patience_ctr += 1

                if frame_patience_ctr >= frame_patience:
                    if len(coordinates_movemenet) != 0:    
                        execution.append(coordinates_movemenet)
                        print("execution collected -> ", len(execution),"[",len(coordinates_movemenet) ,"]")

                        coordinates_movemenet = []

            cv2.imshow("Recording Face and Hand", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()  
    return execution

def text_encode(coordinates_data):
    with open(file_path, "w") as file:
        for execution in coordinates_data:
            file.write("START\n")
            for movement_coordinates in execution:
                for coordinate in movement_coordinates:
                    file.write(str(coordinate))
                    file.write("|")
                file.write("\n")
            file.write("END\n")


def text_decode(file_path):
    coordinates_data = []
    with open(file_path, "r") as file:
        lines = file.readlines()

    execution = []
    for line in lines:
        line = line.strip()
        if line == "START":
            execution = []
        elif line == "END":
            coordinates_data.append(execution)
        else:
            str_coords = line.split("|")[:-1]
            float_coords = [float(coord) for coord in str_coords]
            execution.append(float_coords)

    return coordinates_data



def initialize_dummy_dataset():
    frame_coordinate = []
    movement_coordinate = []
    executions = []

    for x in range(60):
        frame_coordinate.append(0.69)

    for i in range(random.randint(8, 15)):    
        for x in range(random.randint(8, 15)):
            movement_coordinate.append(frame_coordinate)
        executions.append(movement_coordinate)
        movement_coordinate = []

    return executions



In [11]:
def coordinates_visualization(coordintes_dataset):
    canvas_size = 500
    canvas = np.zeros((canvas_size, canvas_size * 2, 3), dtype=np.uint8)

    # for execution in coordintes_dataset:
        # for frame_coordinates in execution[0]:
        #     # for individual_coordinates in frame_coordinates:
        #     for individual_coordinates in range(len(frame_coordinates)):
        #         if individual_coordinates + 1 < len(frame_coordinates):
        #             if individual_coordinates % 2 != 0:
        #                 cv2.circle(canvas, (int(frame_coordinates[individual_coordinates]*1000),int(frame_coordinates[individual_coordinates+1]*1000)), radius=4, color=(0, 255, 0), thickness=-1)
    print(len(coordintes_dataset[0]))
    for frame_coordinates in coordintes_dataset[0]:
        # for individual_coordinates in frame_coordinates:
        for individual_coordinates in range(len(frame_coordinates)):
            if individual_coordinates + 1 < len(frame_coordinates):
                if individual_coordinates % 2 != 0:
                    cv2.circle(canvas, (int(frame_coordinates[individual_coordinates]*1000),int(frame_coordinates[individual_coordinates+1]*1000)), radius=4, color=(0, 255, 0), thickness=-1)

    cv2.imshow("Painted Coordinates", canvas)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return 0

In [5]:
# text_encode(pose_estimation())

In [ ]:

coordinates_visualization(text_decode(file_path))

2
